In [ ]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as vutils
from glob import glob
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import h5py
from tqdm import tqdm
import pandas as pd
import idx2numpy
import torchsummary
import seaborn as sns
from numpyencoder import NumpyEncoder
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import gc
import warnings
import json
warnings.filterwarnings("ignore")
import torch.nn as nn
from torchmeta.modules import (MetaModule, MetaSequential, MetaConv2d,
                               MetaBatchNorm2d, MetaLinear)

import os,time
import torch
from tqdm import tqdm
from torchmeta.transforms import Categorical, Rotation
from torchmeta.utils.data import BatchMetaDataLoader
from torchvision.transforms import ToTensor, Resize, Compose
from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader
import torch.nn.functional as F
from torchmeta.utils.gradient_based import gradient_update_parameters
from torchmeta.utils.prototype import get_prototypes, prototypical_loss

from torch.utils.tensorboard import SummaryWriter

gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
def weights_init(w):
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('bn') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)

In [ ]:
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_cifar_10_data(data_dir, negatives=False):
    """
    Return train_data, train_filenames, train_labels, test_data, test_filenames, test_labels
    """

    # get the meta_data_dict
    # num_cases_per_batch: 1000
    # label_names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # num_vis: :3072

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    # training data
    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    # cifar_train_data_dict
    # 'batch_label': 'training batch 5 of 5'
    # 'data': ndarray
    # 'filenames': list
    # 'labels': list

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, cifar_train_labels, \
        cifar_test_data, cifar_test_filenames, cifar_test_labels, cifar_label_names

In [ ]:
class cifar10dataset(Dataset):
    def __init__(self, cifar_10_dir, tr = True, num_classes=10):
        train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = load_cifar_10_data(cifar_10_dir)
        if(tr):
            self.X = train_data
            self.y = train_labels
        else:
            self.X = test_data
            self.y = test_labels
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
def encode_labels(labels, num_classes):
    train_labels = labels
    if(num_classes==2):
            train_labels[torch.where(((train_labels>1) & (train_labels<8)))[0]] = 2
            train_labels[train_labels!=2] = 0
            train_labels[train_labels!=0] = 1
    elif(num_classes==5):
            train_labels[torch.where(train_labels==2)[0]] = 0
            train_labels[torch.where(train_labels==7)[0]] = 1
            train_labels[torch.where(((train_labels==8) | (train_labels==6)))[0]] = 2
            train_labels[torch.where(((train_labels==9) | (train_labels==5)))[0]] = 4
            train_labels[torch.where(train_labels==4)[0]] = 3
            #train_labels = -1*(train_labels+1)
    return train_labels

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = stride, padding = 0),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 2, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv3 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 1, stride = stride, padding = 0),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.skipconv = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 2, padding = 1, bias=False),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.bn = nn.BatchNorm2d(out_channels)
        
        
    def forward(self, x):
        out = self.conv1(x)
        try:
            out = self.conv2(out)
            out = self.conv3(out)
        except ValueError:
            pass
        res = self.bn(self.skipconv(x))
        return out+res

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, imsize, num_classes = 10, mnist = True):
        super(ResNet, self).__init__()
        self.imszie = imsize
        self.mnist = mnist
        self.layer1 = ResidualBlock(3, 2**4)
        self.layer2 = ResidualBlock(2**4, 2**5)
        self.layer3 = ResidualBlock(2**5, 2**6)
        self.layer4 = ResidualBlock(2**6, 2**7)
        self.layer5 = ResidualBlock(2**7, 2**8)
        self.layer6 = ResidualBlock(2**8, 2**9)
        self.layer7 = ResidualBlock(2**9, 2**10)
        self.imsize = imsize
        self.mutlihead = False
        self.fc2 = nn.Linear(2**8, 2)
        self.fc5 = nn.Linear(2**8, 5)
        self.fc = nn.Linear(2**8, num_classes)
        self.fc_2 = None

    def change_classes(self, classes):
        model = self
        model.multihead = True
        for param in list(model.parameters())[:-6]:
            param.requires_grad = False
        if(classes==2):
            model.fc_2 = model.fc2
        elif(classes==5):
            model.fc_2 = model.fc5
        return model
            
    def encode(self,x):
        bsize = x.shape[0]
        x = x.view(bsize,3,self.imsize, self.imsize)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

    def multihead_forward(self,x):
        bsize = x.shape[0]
        x = x.view(bsize,3,self.imsize, self.imsize)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return F.softmax(self.fc(x.squeeze())), F.softmax(self.fc_2(x.squeeze()))
    def forward(self, x):
        bsize = x.shape[0]
        x = x.view(bsize,3,self.imsize, self.imsize)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        if(self.fc_2):
            return F.softmax(self.fc(x.squeeze())), F.softmax(self.fc_2(x.squeeze()))
        return F.softmax(self.fc(x.squeeze()))

In [ ]:
def eval(model, dataloader, bsize=3000, path=None):
    true = torch.empty(0).to(device)
    pred = torch.empty(0).to(device)
    for ip,t in dataloader:
        #ip = torch.cuda.FloatTensor(data.X)
        #true = data.y#.cpu().detach().numpy()
        op = model(ip.float().to(device))
        p = torch.argmax(op, axis = 1)
        true = torch.cat((true, t.to(device)), dim=0)
        pred = torch.cat((pred, p.to(device)), dim=0)
        del(op, ip)
        torch.cuda.empty_cache()
        gc.collect()
    true = true.cpu().detach().numpy()
    pred = pred.cpu().detach().numpy()
    if path:
        x = {'acc' : accuracy_score(true, pred), 'f1' : precision_recall_fscore_support(true, pred, average='macro')[2], 'misses':list(np.where(pred!=true)[0])}
        y = json.dumps(x, indent=4, cls=NumpyEncoder)
        with open(path, 'w') as outfile:
            outfile.write(y)
    
    return accuracy_score(true, pred), precision_recall_fscore_support(true, pred, average='macro')[2], np.where(pred!=true)[0]

In [ ]:
def plot_tsne(model, dataloader, multihead=False, path=None):
    embdeddings = torch.empty((1,10)).to(device)
    true = torch.empty(0).to(device)
    for ip,t in dataloader:
        if(multihead):
            op,_ = model(ip.float().to(device)).view(ip.shape[0],10)
        else:
            op = model(ip.float().to(device)).view(ip.shape[0],10)
        embdeddings = torch.cat((embdeddings, op.to(device)), dim=0)
        true = torch.cat((true, t.to(device)), dim=0)
        del(op, ip)
        torch.cuda.empty_cache()
        gc.collect()
    embdeddings = embdeddings[1:].cpu().detach().numpy()
    X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(embdeddings)
    plt.figure(figsize=(16,10))
    rndperm = np.random.permutation(dataloader.__len__())
    df = pd.DataFrame(X_embedded,columns=['tsne-one', 'tsne-two'])
    df['y'] = true.cpu().detach().numpy()
    df['label'] = df['y'].apply(lambda i: str(i))
    sns.scatterplot(
    x="tsne-one", y="tsne-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.3
    )
    if path:
        pass

In [ ]:
seed = 369
random.seed(seed)
torch.manual_seed(seed)
print("Random Seed: ", seed)

params = {
    "bsize" : 10000,# Batch size during training.
    'imsize' : 32,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nepochs' : 30,#Number of training epochs.
    'lr' : 0.002,#Learning rate for optimizers
    'nclasses' : 10, #number of classes
    'save_epoch' : 20}# Save step.

device = torch.device("cuda:0" if(torch.cuda.is_available()) else "cpu")
#torch.device('cpu')#
cifar10 = cifar10dataset('data')
cifar10TrainLoader = DataLoader(cifar10, params['bsize'], shuffle=True)
cifar10Test = cifar10dataset('data', False)
cifar10TestLoader = DataLoader(cifar10Test, params['bsize'], shuffle=True)

In [ ]:
model = ResNet(params['imsize']).to(device)
model.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])  
train_loss, test_accuracies = [],[]
for epoch in range(params['nepochs']):
    for images, labels in cifar10TrainLoader:  
        images = images.to(device).float()
        labels = labels.type(torch.LongTensor).to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    test_accuracies.append(eval(model, cifar10TestLoader)[0])
    print ('Epoch [{}/{}], Loss: {:.4f}, Test accuracy : {:.4}' 
                   .format(epoch+1, params['nepochs'], loss.item(), test_accuracies[-1]))
torch.cuda.empty_cache()
gc.collect()
eval(model, cifar10TestLoader, path='outputs/cifar10.json')
torch.save({
            'model' : model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            'params' : params
            }, 'outputs/cifar10.pth')

In [ ]:
plt.plot(np.arange(len(train_loss)), np.array(train_loss))
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Iteration vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(len(test_accuracies)), np.array(test_accuracies))
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Accuracy vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plot_tsne(model, cifar10TestLoader)

In [ ]:
model5 = model.change_classes(5)

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model5.parameters(), lr=params['lr'])  
train_loss, test_accuracies = [],[]
for epoch in range(params['nepochs']):
    for images, labels in cifar10TrainLoader:  
        images = images.to(device).float()
        labels = labels.type(torch.LongTensor).to(device)
        labels2 = encode_labels(labels,5)
        #lebels = -labels
        outputs, outputs2 = model5.multihead_forward(images)
        loss = criterion(outputs, labels) + \
            criterion(outputs2, labels2)
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        train_loss.append(loss.item())
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    test_accuracies.append(eval(model5, cifar10TestLoader)[0])
    print ('Epoch [{}/{}], Loss: {:.4f}, Test accuracy : {:.4}' 
                   .format(epoch+1, params['nepochs'], loss.item(), test_accuracies[-1]))
torch.cuda.empty_cache()
gc.collect()
eval(model5, cifar10TestLoader, path='outputs/cifar5.json')
torch.save({
            'model' : model5.state_dict(),
            'optimizer' : optimizer.state_dict(),
            'params' : params
            }, 'outputs/cifar5.pth')

In [ ]:
plt.plot(np.arange(len(train_loss)), np.array(train_loss))
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Iteration vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(len(test_accuracies)), np.array(test_accuracies))
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Accuracy vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plot_tsne(model5, cifar10TestLoader)

In [ ]:
model2 = model.change_classes(2)

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model5.parameters(), lr=params['lr'])  
train_loss, test_accuracies = [],[]
for epoch in range(params['nepochs']):
    for images, labels in cifar10TrainLoader:  
        images = images.to(device).float()
        labels = labels.type(torch.LongTensor).to(device)
        labels2 = encode_labels(labels,2)
        outputs, outputs2 = model2.multihead_forward(images)
        loss = criterion(outputs, labels) + \
            criterion(outputs2, labels2)
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        train_loss.append(loss.item())
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    test_accuracies.append(eval(model2, cifar10TestLoader)[0])
    print ('Epoch [{}/{}], Loss: {:.4f}, Test accuracy : {:.4}' 
                   .format(epoch+1, params['nepochs'], loss.item(), test_accuracies[-1]))
torch.cuda.empty_cache()
gc.collect()
eval(model2, cifar10TestLoader, path='outputs/cifar5.json')
torch.save({
            'model' : model2.state_dict(),
            'optimizer' : optimizer.state_dict(),
            'params' : params
            }, 'outputs/cifar2.pth')

In [ ]:
plt.plot(np.arange(len(train_loss)), np.array(train_loss))
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Iteration vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(len(test_accuracies)), np.array(test_accuracies))
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Accuracy vs loss(resnet trained on Clipart)")
plt.legend()
plt.show()

In [ ]:
plot_tsne(model2, cifar10TestLoader)

In [ ]:
parser = argparse.ArgumentParser(description='Train MoCo on CIFAR-10')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.06, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=200, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[120, 160], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=512, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

# moco specific configs:
parser.add_argument('--moco-dim', default=128, type=int, help='feature dimension')
parser.add_argument('--moco-k', default=2048, type=int, help='queue size; number of negative keys')
parser.add_argument('--moco-m', default=0.99, type=float, help='moco momentum of updating key encoder')
parser.add_argument('--moco-t', default=0.1, type=float, help='softmax temperature')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=200, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.1, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 200
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = './cache-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-moco")

print(args)

In [ ]:
class CIFAR10Pair(CIFAR10):
    """CIFAR10 Dataset.
    """
    def __getitem__(self, index):
        img = self.data[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

memory_data = CIFAR10(root='data', train=True, transform=test_transform, download=True)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True)


In [ ]:
# SplitBatchNorm: simulate multi-gpu behavior of BatchNorm in one gpu by splitting alone the batch dimension
# implementation adapted from https://github.com/davidcpage/cifar10-fast/blob/master/torch_backend.py
class SplitBatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        
    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            running_mean_split = self.running_mean.repeat(self.num_splits)
            running_var_split = self.running_var.repeat(self.num_splits)
            outcome = nn.functional.batch_norm(
                input.view(-1, C * self.num_splits, H, W), running_mean_split, running_var_split, 
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
            self.running_mean.data.copy_(running_mean_split.view(self.num_splits, C).mean(dim=0))
            self.running_var.data.copy_(running_var_split.view(self.num_splits, C).mean(dim=0))
            return outcome
        else:
            return nn.functional.batch_norm(
                input, self.running_mean, self.running_var, 
                self.weight, self.bias, False, self.momentum, self.eps)

class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=128, arch=None, bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

In [ ]:
class ModelMoCo(nn.Module):
    def __init__(self, dim=128, K=4096, m=0.99, T=0.1, arch='resnet18', bn_splits=8, symmetric=True):
        super(ModelMoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.symmetric = symmetric

        # create the encoders
        self.encoder_q = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)
        self.encoder_k = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.t()  # transpose
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_single_gpu(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        """
        # random shuffle index
        idx_shuffle = torch.randperm(x.shape[0]).cuda()

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        return x[idx_shuffle], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_single_gpu(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        """
        return x[idx_unshuffle]

    def contrastive_loss(self, im_q, im_k):
        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)  # already normalized

        # compute key features
        with torch.no_grad():  # no gradient to keys
            # shuffle for making use of BN
            im_k_, idx_unshuffle = self._batch_shuffle_single_gpu(im_k)

            k = self.encoder_k(im_k_)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)  # already normalized

            # undo shuffle
            k = self._batch_unshuffle_single_gpu(k, idx_unshuffle)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()
        
        loss = nn.CrossEntropyLoss().cuda()(logits, labels)

        return loss, q, k

    def forward(self, im1, im2):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            loss
        """

        # update the key encoder
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()

        # compute loss
        if self.symmetric:  # asymmetric loss
            loss_12, q1, k2 = self.contrastive_loss(im1, im2)
            loss_21, q2, k1 = self.contrastive_loss(im2, im1)
            loss = loss_12 + loss_21
            k = torch.cat([k1, k2], dim=0)
        else:  # asymmetric loss
            loss, q, k = self.contrastive_loss(im1, im2)

        self._dequeue_and_enqueue(k)

        return loss

# create model
model = ModelMoCo(
        dim=args.moco_dim,
        K=args.moco_k,
        m=args.moco_m,
        T=args.moco_t,
        arch=args.arch,
        bn_splits=args.bn_splits,
        symmetric=args.symmetric,# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    ).cuda()
print(model.encoder_q)

In [ ]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# test using a knn monitor
def test(net, memory_data_loader, test_data_loader, epoch, args):
    net.eval()
    classes = len(memory_data_loader.dataset.classes)
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        # generate feature bank
        for data, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature = net(data.cuda(non_blocking=True))
            feature = F.normalize(feature, dim=1)
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature = net(data)
            feature = F.normalize(feature, dim=1)
            
            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, args.knn_k, args.knn_t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100

# knn monitor as in InstDisc https://arxiv.org/abs/1805.01978
# implementation follows http://github.com/zhirongw/lemniscate.pytorch and https://github.com/leftthomas/SimCLR
def knn_predict(feature, feature_bank, feature_labels, classes, knn_k, knn_t):
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / knn_t).exp()

    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)

    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

In [ ]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.encoder_q, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

In [ ]:
def meta_conv3x3(in_channels, out_channels, **kwargs):
    return MetaSequential(
        MetaConv2d(in_channels, out_channels, kernel_size=3, padding=1, **kwargs),
        MetaBatchNorm2d(out_channels, momentum=1., track_running_stats=False),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
def conv3x3(in_channels, out_channels, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, **kwargs),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )


In [ ]:
class ConvolutionalNeuralNetwork(MetaModule):
    def __init__(self, in_channels, out_features, hidden_size=32):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.in_channels = in_channels
        self.out_features = out_features
        self.hidden_size = hidden_size

        self.features = MetaSequential(
            meta_conv3x3(in_channels, hidden_size),
            meta_conv3x3(hidden_size, hidden_size),
            meta_conv3x3(hidden_size, hidden_size),
            meta_conv3x3(hidden_size, hidden_size),
            MetaSequential(MetaConv2d(hidden_size, hidden_size, kernel_size=3, padding=1),
                           MetaBatchNorm2d(hidden_size, momentum=1., track_running_stats=False),nn.ReLU()))

        self.classifier = MetaLinear(hidden_size, out_features)

    def forward(self, inputs, params=None):
        features = self.features(inputs, params=self.get_subdict(params, 'features'))
        features = features.view((features.size(0), -1))
        logits = self.classifier(features, params=self.get_subdict(params, 'classifier'))
        return logits


class PrototypicalNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_size=32):
        super(PrototypicalNetwork, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_size = hidden_size

        self.encoder = nn.Sequential(
            conv3x3(in_channels, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size),
            nn.Sequential(nn.Conv2d(hidden_size, out_channels, kernel_size=3, padding=1),nn.BatchNorm2d(out_channels),nn.ReLU())
        )



    def forward(self, inputs):
        embeddings = self.encoder(inputs.view(-1, *inputs.shape[2:]))
        return embeddings.view(*inputs.shape[:2], -1)

In [ ]:
def get_accuracy(logits, targets):
   
    _, predictions = torch.max(logits, dim=-1)
    return torch.mean(predictions.eq(targets).float())

def print_log(log_path,eps_no,accuracy,loss):
    with open(log_path,'a') as file:
        print(f'Episode Number = {eps_no} Accuracy = {accuracy} Loss = {loss}',file= file)

In [ ]:
def get_dataloader(config):
    num_ways,num_shots,batch_size = config['num-ways'],config['num-shots'],config['batch_size']
    
    dataset = omniglot('data', 
                        ways = num_ways, 
                        shots = num_shots, 
                        test_shots = num_shots, 
                        meta_train = True, 
                        transform=Compose([Resize(28), ToTensor()]),
                        target_transform=Categorical(num_classes=num_ways),
                        class_augmentations=[Rotation([90, 180, 270])],
                        download = True)
    
    dataloader = BatchMetaDataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1)
    return dataloader

In [ ]:
def eval_maml(model,dataloader,device):
    num_batches,batch_size =1000,16
    model.eval()
    with torch.no_grad():
        with tqdm(dataloader, total=num_batches) as pbar:
            for batch_idx, batch in enumerate(pbar):
                model.zero_grad()

                test_inputs, test_targets = batch['test']
                test_inputs = test_inputs.to(device=device)
                test_targets = test_targets.to(device=device)

                outer_loss = torch.tensor(0., device=device)
                accuracy = torch.tensor(0., device=device)
                for task_idx, (test_input,
                        test_target) in enumerate(zip(test_inputs, test_targets)):
                    

                    test_logit = model(test_input, params=model.parameters())
                    outer_loss += F.cross_entropy(test_logit, test_target)

                    accuracy += get_accuracy(test_logit, test_target)

                outer_loss.div_(batch_size)
                accuracy.div_(batch_size)
                
                pbar.set_postfix(accuracy='{0:.4f}'.format(accuracy.item()))
                
                if batch_idx >= num_batches:
                    break

    
    final_accuracy = final_accuracy/num_batches
    return final_accuracy

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train_maml(config):
    

    device = 'cuda:1'
    num_ways,num_shots,hidden_size,batch_size = config['num-ways'],config['num-shots'],config['hidden-size'],config['batch_size']
    num_batches,output_folder,runs_dir = config['num-batches'],config['output_folder'],config['tensorboard_runs']
    log_path = config['logs']
    dataloader =  get_dataloader(config)
    writer = SummaryWriter(runs_dir)
    model = ConvolutionalNeuralNetwork(1,
                                       num_ways,
                                       hidden_size=hidden_size)
    model.to(device=device)
    num_params = count_parameters(model)
    print(num_params)
    
    
    final_accuracy = 0.0
    
    # Training loop
    step = 0

    
    meta_optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Training loop
    with tqdm(dataloader, total=num_batches) as pbar:
        for batch_idx, batch in enumerate(pbar):
            model.train()
            model.zero_grad()

            train_inputs, train_targets = batch['train']
            train_inputs = train_inputs.to(device=device)
            train_targets = train_targets.to(device=device)

            test_inputs, test_targets = batch['test']
            test_inputs = test_inputs.to(device=device)
            test_targets = test_targets.to(device=device)

            outer_loss = torch.tensor(0., device=device)
            accuracy = torch.tensor(0., device=device)
            for task_idx, (train_input, train_target, test_input,
                    test_target) in enumerate(zip(train_inputs, train_targets,
                    test_inputs, test_targets)):
                train_logit = model(train_input)
                inner_loss = F.cross_entropy(train_logit, train_target)

                model.zero_grad()
                params = gradient_update_parameters(model,
                                                    inner_loss,
                                                    step_size=0.4,
                                                    first_order=True)

                test_logit = model(test_input, params=params)
                outer_loss += F.cross_entropy(test_logit, test_target)

                with torch.no_grad():
                    accuracy += get_accuracy(test_logit, test_target)

            outer_loss.div_(batch_size)
            accuracy.div_(batch_size)
            writer.add_scalar('Loss',outer_loss.item(),global_step=step)
            writer.add_scalar('Accuracy',accuracy.item(),global_step=step)
            step += 1
            outer_loss.backward()
            meta_optimizer.step()

            pbar.set_postfix(accuracy='{0:.4f}'.format(accuracy.item()))
            print_log(log_path,batch_idx,accuracy.item(),outer_loss.item())
            if batch_idx >= num_batches:
                break

    # Save model
    if output_folder is not None:
        filename = os.path.join(output_folder, 'maml_omniglot_'
            '{0}shot_{1}way.th'.format(num_shots, num_ways))
        with open(filename, 'wb') as f:
            state_dict = model.state_dict()
            torch.save(state_dict, f)
    final_accuracy = eval_maml(model,dataloader,device=device)
    with open(log_path,'a') as file:
        print(f'Average Test Accuracy = {final_accuracy}',file= file)

In [ ]:
def get_accuracy_proto(prototypes, embeddings, targets):
    sq_distances = torch.sum((prototypes.unsqueeze(1)
        - embeddings.unsqueeze(2)) ** 2, dim=-1)
    _, predictions = torch.min(sq_distances, dim=-1)
    return torch.mean(predictions.eq(targets).float())

def print_log(log_path,eps_no,accuracy,loss):
    with open(log_path,'a') as file:
        print(f'Episode Number = {eps_no} Accuracy = {accuracy} Loss = {loss}',file= file)

In [ ]:
def eval_proto(model,dataloader,device):
    num_batches,num_ways =1000,5
    model.eval()
    final_accuracy = 0.0
    with torch.no_grad():
        with tqdm(dataloader, total=1000) as pbar:
            for batch_idx, batch in enumerate(pbar):
                    model.zero_grad()

                    train_inputs, train_targets = batch['train']
                    train_inputs = train_inputs.to(device=device)
                    train_targets = train_targets.to(device=device)
                    train_embeddings = model(train_inputs)

                    test_inputs, test_targets = batch['test']
                    test_inputs = test_inputs.to(device=device)
                    test_targets = test_targets.to(device=device)
                    test_embeddings = model(test_inputs)

                    prototypes = get_prototypes(train_embeddings, train_targets,num_ways)
                    loss = prototypical_loss(prototypes, test_embeddings, test_targets)
                    
                    accuracy = get_accuracy(prototypes, test_embeddings, test_targets)
                    pbar.set_postfix(accuracy='{0:.4f}'.format(accuracy.item()))
                
                    final_accuracy += accuracy
                    
                    if batch_idx >= num_batches: 
                        break
    final_accuracy = final_accuracy/num_batches
    return final_accuracy

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_proto(config):

    device = 'cuda:1'
    num_ways,num_shots,hidden_size = config['num-ways'],config['num-shots'],config['hidden-size']
    num_batches,output_folder,runs_dir = config['num-batches'],config['output_folder'],config['tensorboard_runs']
    embedding_size,log_path = config['embedding_size'],config['logs']
    dataloader =  get_dataloader(config)
    writer = SummaryWriter(runs_dir)
    model = PrototypicalNetwork(1,
                                embedding_size,
                                hidden_size=hidden_size)
    num_params = count_parameters(model)
    print(num_params)
    
    model.to(device=device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    final_accuracy = 0.0
    
    # Training loop
    step = 0
    with tqdm(dataloader, total=num_batches) as pbar:
        for batch_idx, batch in enumerate(pbar):
            model.zero_grad()

            train_inputs, train_targets = batch['train']
            train_inputs = train_inputs.to(device=device)
            train_targets = train_targets.to(device=device)
            train_embeddings = model(train_inputs)

            test_inputs, test_targets = batch['test']
            test_inputs = test_inputs.to(device=device)
            test_targets = test_targets.to(device=device)
            test_embeddings = model(test_inputs)

            prototypes = get_prototypes(train_embeddings, train_targets,num_ways)
            loss = prototypical_loss(prototypes, test_embeddings, test_targets)
            
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                accuracy = get_accuracy_proto(prototypes, test_embeddings, test_targets)
                pbar.set_postfix(accuracy='{0:.4f}'.format(accuracy.item()))
            writer.add_scalar('Prototypical Loss',loss.item(),global_step=step)
            writer.add_scalar('Accuracy',accuracy.item(),global_step=step)
            step += 1
            print_log(log_path,batch_idx,accuracy.item(),loss.item())
            if batch_idx >= num_batches: 
                break
            

    # Save model
    if output_folder is not None:
        filename = os.path.join(output_folder, 'protonet_omniglot_'
            '{0}shot_{1}way.pt'.format(num_shots, num_ways))
        with open(filename, 'wb') as f:
            state_dict = model.state_dict()
            torch.save(state_dict, f)
    final_accuracy = eval_proto(model,dataloader,device=device)
    with open(log_path,'a') as file:
        print(f'Average Test Accuracy = {final_accuracy}',file= file)

In [ ]:

config = {'folder':'./data/omniglot','num-shots':5,'num-ways':5,'step-size':0.4, 'embedding_size':32,\
        'hidden-size':32,'batch_size':16,'num-batches':1000,'tensorboard_runs':'runs-proto/n=5_k=5/','logs':'./proto_n=5_k=5_logs.txt','output_folder':'./data/'}
start_time = time.time()
train_proto(config)
end_time = time.time()

with open(config['logs'] ,'a') as f:
    print(f'ProtoNet:Time taken for 5 way 5 shot ={end_time-start_time}',file = f)

config = {'folder':'./data/omniglot','num-shots':1,'num-ways':5,'step-size':0.4, 'embedding_size':32,\
        'hidden-size':32,'batch_size':16,'num-batches':1000,'tensorboard_runs':'runs-proto/n=5_k=1/','logs':'./proto_n=5_k=1_logs.txt','output_folder':'./data/'}


start_time = time.time()
train_proto(config)
end_time = time.time()
with open(config['logs'] ,'a') as f:
    print(f'ProtoNet:Time taken for 5 way 1 shot ={end_time-start_time}',file = f)

In [ ]:
config = {'folder':'./data/omniglot','num-shots':5,'num-ways':5,'step-size':0.4,\
        'hidden-size':32,'batch_size':16,'num-batches':1000,'tensorboard_runs':'runs-maml/n=5_k=5/','logs':'./maml_new_n=5_k=5_logs.txt','output_folder':'./data/'}
start_time = time.time()
train_maml(config)
end_time = time.time()

with open(config['logs'] ,'a') as f:
    print(f'MAML:Time taken for 5 way 5 shot ={end_time-start_time}',file = f)

config = {'folder':'./data/omniglot','num-shots':1,'num-ways':5,'step-size':0.4,\
        'hidden-size':32,'batch_size':16,'num-batches':1000,'tensorboard_runs':'runs-maml/n=5_k=1/','logs':'./maml_new_n=5_k=1_logs.txt','output_folder':'./data/'}


start_time = time.time()
train_maml(config)
end_time = time.time()
with open(config['logs'] ,'a') as f:
    print(f'MAML:Time taken for 5 way 1 shot ={end_time-start_time}',file = f)